In [33]:
import json
import os

import inro.emme.datatable as _dt
import inro.emme.desktop.worksheet as _ws
import pandas as _pd

mm = inro.modeller.Modeller()
desktop = mm.desktop

In [34]:
# modeller tools used
create_attribute = mm.tool('inro.emme.data.extra_attribute.create_extra_attribute')
network_calculator = mm.tool('inro.emme.network_calculation.network_calculator')


In [35]:
# names of attributes used (existing and to be created)

x_alight = '@alight_per_door'
x_board = '@board_per_door'
y_dwt = '@dwtout'
f_crowd = '@crowding_factor'

In [36]:
# create attributes

# function to create extra attributes for transit segments
def create_att_segment( att, desc):
    create_attribute(extra_attribute_type='TRANSIT_SEGMENT',
                 extra_attribute_name=att,
                 extra_attribute_description=desc,
                 overwrite=True)
    return

# attributes to create and their descriptions
new_atts = {y_dwt:'calculated dwt from assigned network',
            x_board:'boardings per door for boarding',
            x_alight:'alightings per door for alighting'
            }

#create the attributes
for att in new_atts:
    create_att_segment(att, new_atts[att])

In [37]:
# builds network calculation spec for transit segments

def net_calc_segment_spec(expression, result_att, transit_line_filter):
    net_calc_spec = {
        "type" : "NETWORK_CALCULATION",
        "result" : result_att,
        "expression" : expression,
        "selections" : {
            "transit_line" : transit_line_filter,
            "link" : "all"}}
    
    return net_calc_spec

In [38]:
# get vehicle group specs
# contains groupings, with vehicle types listed, number of doors, and whether boardings and alightings use separate doors
with open('specs.json') as specs:
    vehicle_grouping = json.load(specs)['vehicle_grouping']
    

In [39]:
# calculate boardings and alightings per door based on door usage in vehicle group specs
board_alight_specs = []

for group in vehicle_grouping:
    vehicle_types = vehicle_grouping[group]['vehicle types']
    
    # concatenate expression string for vehicle types
    veh_str = ''
    for veh in vehicle_types:
        veh_str += 'vehicle = ' + str(veh)
        if veh != vehicle_types[-1]:
            veh_str += ' | '
    
    doors = vehicle_grouping[group]['doors']
    
    # differentiate beween separate doors vs all doors for boarding and alighting
    separate_board = vehicle_grouping[group]['separate_board']

    if separate_board > 0:
        doors_board = separate_board
        doors_alight = doors - separate_board
        
    else:
        doors_board = doors
        doors_alight = doors

    # network calculator specs for boardings and alightings per door
    alight_spec = net_calc_segment_spec('@alightings_avg / {}'.format(doors_alight), x_alight, veh_str)
    board_spec = net_calc_segment_spec('@boardings_avg / {}'.format(doors_board), x_board, veh_str) 
    board_alight_specs.extend([alight_spec, board_spec])
    
#     network_calculator([alight_spec, board_spec])

network_calculator(board_alight_specs)

[{'average': 0.871694,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 37.83696,
  'maximum_at': {'i_node': 42467, 'j_node': 41681, 'transit_line': u'GB36Bw'},
  'minimum': 0.0,
  'minimum_at': {'i_node': 10709, 'j_node': 11062, 'transit_line': u'GB36Bw'},
  'num_attributes_changed': 6,
  'num_evaluations': 82,
  'num_results_obtained': 82,
  'result_type': 'TRANSIT_SEGMENT',
  'sum': 71.478882},
 {'average': 1.285968,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 68.579247,
  'maximum_at': {'i_node': 10709, 'j_node': 11062, 'transit_line': u'GB36Bw'},
  'minimum': 0.0,
  'minimum_at': {'i_node': 11062, 'j_node': 11794, 'transit_line': u'GB36Bw'},
  'num_attributes_changed': 7,
  'num_evaluations': 82,
  'num_results_obtained': 82,
  'result_type': 'TRANSIT_SEGMENT',
  'sum': 105.449402},
 {'average': 1.036093,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 168.152206,
  'maximum_at': {'i_node': 60023, 'j_node': 60767, 'transit_line': u'GB47Zw'},
  'minimum': -3.1e-05

In [40]:
# calculate dwell times

# model parameters
crowd_f_lim = 1.0 # lower limit for crowding factor
c0 = 7.0
c1 = 2.0
c2 = 3.0

filter_exp = '(@boardings_avg>=1 || @alightings_avg>=1)'

# apply formula depending on whether boardings and alightings use separate doors.
dwt_specs = []
for group in vehicle_grouping:
    if separate_board > 0:
        dwt_expression = '{}*({} + ({}.max.{})*(({}*{}).max.({}*{})))'.format(filter_exp,c0,crowd_f_lim, f_crowd, c1, x_alight, c2, x_board)
    else:
        dwt_expression = '{}*({} + ({}.max.{})*(({}*{})+({}*{})))'.format(filter_exp,c0,crowd_f_lim, f_crowd, c1, x_alight, c2, x_board)
    
    dwt_specs.append(net_calc_segment_spec(dwt_expression, y_dwt, 'all'))
    
network_calculator(dwt_specs)

[{'average': 30.826937,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 3901.428711,
  'maximum_at': {'i_node': 10046, 'j_node': 10039, 'transit_line': u'T044Ba'},
  'minimum': 0.0,
  'minimum_at': {'i_node': 40493, 'j_node': 40492, 'transit_line': u'B001B'},
  'num_attributes_changed': 31482,
  'num_evaluations': 80096,
  'num_results_obtained': 80096,
  'result_type': 'TRANSIT_SEGMENT',
  'sum': 2469114.25},
 {'average': 30.826937,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 3901.428711,
  'maximum_at': {'i_node': 10046, 'j_node': 10039, 'transit_line': u'T044Ba'},
  'minimum': 0.0,
  'minimum_at': {'i_node': 40493, 'j_node': 40492, 'transit_line': u'B001B'},
  'num_attributes_changed': 0,
  'num_evaluations': 80096,
  'num_results_obtained': 80096,
  'result_type': 'TRANSIT_SEGMENT',
  'sum': 2469114.25},
 {'average': 30.826937,
  'expression_type': 'TRANSIT_SEGMENT',
  'maximum': 3901.428711,
  'maximum_at': {'i_node': 10046, 'j_node': 10039, 'transit_line': u'T044Ba'

In [41]:
# create data tables of transit segment attributes and taz, join tables

project_path = os.path.dirname(desktop.project.path)
project_table_db = desktop.project.data_tables()

### TAZs
# import TAZ shp

taz_path = os.path.join(project_path, 'Scripts\MX_Task20\GGHMv4_TAZ_updated\GGHMv4_TAZ_updated.shp').replace("\\", "/")
taz_ds = _dt.DataSource(taz_path)
taz_data = taz_ds.layer('GGHMv4_TAZ_updated').get_data()

# remove all attributes except TAZ_NO
att_delete = []
taz_att = taz_data.attributes()
for att in taz_att:
    if att.name != 'TAZ_NO' and att.atype != 'GEOMETRY':
        att_delete.append(att.name)

for att in att_delete:
    taz_data.delete_attribute(att)

# create table out of data
project_table_db.create_table('TAZ_shp', taz_data, overwrite = True)


### Transit Segment
# create transit segment data table from network table

root_worksheet_folder = desktop.root_worksheet_folder()
tseg_table = root_worksheet_folder.find_item(["General", "Network", "Transit", 'Transit segments'])

# remove unnecessary columns
tseg = tseg_table.open()
for i in range(8):
    tseg.delete_column(1)

# create new column for segment id
id_col = _ws.Column()
id_col.name = 'id'
id_col.expression = 'id'
tseg.add_column(0, id_col)

# create new column for modelled dwell times
dwt_col = _ws.Column()
dwt_col.name = 'modelled_dwell'
dwt_col.expression = y_dwt
tseg.add_column(1, dwt_col)

tseg.par('Filter').set('')

# save sheet
tseg.par('Name').set('TransitSegments_DwellOut')
tseg.save_as_data_table('TransitSegments_DwellOut', overwrite = True)
tseg.close()

# gets the TAZ and segment data tables, joins TAZ to segments spatially
project_table_db = desktop.project.data_tables()
taz_dt = project_table_db.table('TAZ_shp')
seg_dt = project_table_db.table('TransitSegments_DwellOut')
joint_dt = seg_dt.spatial_join(taz_dt,
                    predicate = 'INTERSECT')


In [42]:
# convert joined table to data frame and aggregate

# data table to pandas data frame
def to_pandas(dat):
    dfs = []
    for a in dat.attributes():
        if a.atype == 'GEOMETRY':
            dfs.append( _pd.DataFrame( {a.name: [x.text for x in a.values]} ) )
        else:
            dfs.append( _pd.DataFrame( {a.name: a.values} ) )
    df = dfs[0]
    for x in dfs[1:]:
        df = df.join(x)
    return df

df = to_pandas(joint_dt)
df = df[['Line', 'id', 'modelled_dwell', 'TAZ_NO']] # filter to needed columns

# average by line and taz
df_line_taz = df.groupby(['Line','TAZ_NO']).mean()
df_taz = df.groupby(['TAZ_NO']).mean()

In [43]:
# filter and save data frames as csv
df_line_taz = df_line_taz[['modelled_dwell']]
df_taz = df_taz[['modelled_dwell']]
df_line_taz.to_csv(os.path.join(project_path, 'Media','dwell_byLineTAZ.csv'), index = True)
df_taz.to_csv(os.path.join(project_path, 'Media','dwell_byTAZ.csv'), index = True)